# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Steps involved

In [5]:
#1: Read and load the .txt file into an numpy array
#2: Creating an Empty DataFrame
#3: Creating a full matrix
#4: Training the Naive Bayes Model
        # Calculating the probability of spam
        # P(Token|Spam) - Prob of a Token occuring in a Spam email:
        # P(Token|Ham) - Prob of a Token occuring in a Non-Spam email:
        # P(Token) - Prob of a Token
        # Save the trained model in txt file
#5: Prepare the test data

# Constants

In [2]:
VOCAB_SIZE = 2500
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'


# Read and load the .txt file into an numpy array

In [3]:
sparse_train_data= np.loadtxt(TRAINING_DATA_FILE,delimiter=' ',dtype=int)
sparse_test_data= np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [4]:
print('Number of rows in training array:',sparse_train_data.shape[0])
print('Number of rows in test array:',sparse_test_data.shape[0])

Number of rows in training array: 265427
Number of rows in test array: 110522


In [5]:
sparse_train_data[:5]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1]])

In [6]:
print('Number of unique emails in training data:',np.unique(sparse_train_data[:,0]).size)
print('Number of unique emails in test data:',np.unique(sparse_test_data[:,0]).size)

Number of unique emails in training data: 4015
Number of unique emails in test data: 1724


# Creating an Empty DataFrame:

In [7]:
#UNDERSTANDINGS:
#Column names of the Dataframe:
column_names =  ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
column_names[0:7]

['DOC_ID', 'CATEGORY', 0, 1, 2, 3, 4]

In [8]:
#storing the index values which acts as the index of the DF: 
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [9]:
#Creating the DF: 
full_train_data = pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True) #inplace helps to avoid a= a+changes

In [10]:
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#END OF UNDERSTANDINGS

# Creating a full matrix

In [12]:
# The following DF consists of DOC_ID , Category and word ids of all 2500 word, having the nr of occurences as values: 
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    """
    The function collects the position of the Doc index, word index and freq in the Sparse matrix column
    sparse_matrix will be the Training dataset and nr_words will be the VOCAB_SIZE-- Top 2500 freq occuring words
    """
    #Creating an empty sparse matrix with just zeros in it:
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True) 
    
    for i in range(sparse_train_data.shape[0]):
        #Gathering the data in variables:
        doc_nr = sparse_train_data[i][doc_idx]
        word_id = sparse_train_data[i][word_idx]
        label = sparse_train_data[i][cat_idx]
        occurence = sparse_train_data[i][freq_idx]
        
        #Populating the data into the DF from the variables, the .at is used toidentify the cell location:
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurence
        
    full_matrix.set_index('DOC_ID',inplace=True) #Setting the index value
    
    return full_matrix


In [13]:
full_train_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

In [14]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

### Calculating the probability of spam

In [15]:
#Total number of emails:
full_train_data.CATEGORY.size
#Number of spam:
full_train_data.CATEGORY.sum()

1250

In [16]:
prob_spam= full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of Spam mail:',prob_spam)

Probability of Spam mail: 0.31133250311332505


### Total number of Words/Tokens

In [17]:
# Fetching all the columns except category in the full_train_data DF:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,2,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,2,4,2,3,1,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#Calculates the number of words per email:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [19]:
total_wc= email_lengths.sum()
total_wc

445891

### Number of words in Spam and Ham emails:

In [20]:
spam_lengths = email_lengths[full_train_data.CATEGORY==1]
ham_lengths = email_lengths[full_train_data.CATEGORY==0]

In [21]:
#Total number of words in spam emails:
spam_wc = spam_lengths.sum()
#Total number of words in ham emails:
nonspam_wc =  ham_lengths.sum()
#Total number of emails in both:
both = full_train_features.sum().sum()

In [22]:
print('Average number of words in spam emails : {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average number of words in non-spam emails : {:.0f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average number of words in spam emails : 157
Average number of words in non-spam emails : 91


### Summing the spam tokens

In [23]:
#Creating a DF with just spam emails in it:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY==1]

In [24]:
#Summing column wise (calculating the word id and its total occurences):
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1  #Laplace smoothing

### Summing the Ham tokens:

In [25]:
#Creating a DF with just non-spam emails in it:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY==0]

In [26]:
#Summing column wise (calculating the word id and its total occurences):
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1  #Laplace smoothing -- Doubt

# P(Token|Spam) - Prob of a Token occuring in a Spam email:

In [27]:
prob_tokens_spam = summed_spam_tokens/(spam_wc + VOCAB_SIZE)

In [28]:
prob_tokens_spam.sum()

1.0

# P(Token|Ham) - Prob of a Token occuring in a Non-Spam email:

In [29]:
prob_tokens_nonspam = summed_ham_tokens/(nonspam_wc + VOCAB_SIZE)

In [37]:
prob_tokens_nonspam.head()

0    0.020863
1    0.009887
2    0.008063
3    0.003735
4    0.006311
dtype: float64

# P(Token) - Prob of a Token

In [36]:
prob_tokens_all = full_train_features.sum(axis=0)/total_wc
prob_tokens_all.head()

0    0.015842
1    0.007771
2    0.007605
3    0.006820
4    0.006596
dtype: float64

# Save the trained model

In [32]:
#np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
#np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_nonspam)
#np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Prepare the test data

In [43]:
full_test_data = make_full_matrix(sparse_test_data,VOCAB_SIZE)

In [34]:
X_test = full_test_data.loc[:,full_test_data.columns !='CATEGORY'] 
y_test = full_test_data.CATEGORY

In [35]:
#np.savetxt(TEST_TARGET_FILE,y_test)
#np.savetxt(TEST_FEATURE_MATRIX,X_test)

In [39]:
X_test.shape

(4015, 2500)